Processing the Repository Files:
In order to access the files in the target repository, the script will clone the desired repository onto your computer, placing the files in a folder named "repos". Once we download the files, it is a matter of looping through the directory to create a list of files. It is possible to filter out specific extensions or environmental items.

In [7]:
from dotenv import load_dotenv, dotenv_values
import os

# Load environment variables from .env file
config = dotenv_values("C:/Users/SACHENDRA/Documents/Activeloop/.env")
load_dotenv("C:/Users/SACHENDRA/Documents/Activeloop/.env")

True

In [4]:
import os
from langchain_community.document_loaders import TextLoader
root_dir = r"C:\Users\SACHENDRA\Documents\Activeloop\Langchain_and_Vector_Databases_in_Production\6_Giving_memory_to_LLMs\rest-api-server-java-sdk"
docs = []
file_extensions = []

for dirpath, dirnames, filenames in os.walk(root_dir):
    for file in filenames:
        file_path = os.path.join(dirpath, file)
        if file_extensions and os.path.splitext(file)[1] not in file_extensions:
            continue
        try:
            loader = TextLoader(file_path, encoding="utf-8")
            docs.extend(loader.load_and_split())
        except UnicodeDecodeError:
            print(f"UnicodeDecodeError: Skipping {file_path}")
            continue
        except Exception as e:
            print(f"Error loading {file_path}: {e}")

Error loading C:\Users\SACHENDRA\Documents\Activeloop\Langchain_and_Vector_Databases_in_Production\6_Giving_memory_to_LLMs\rest-api-server-java-sdk\jacoco.exec: Error loading C:\Users\SACHENDRA\Documents\Activeloop\Langchain_and_Vector_Databases_in_Production\6_Giving_memory_to_LLMs\rest-api-server-java-sdk\jacoco.exec
Error loading C:\Users\SACHENDRA\Documents\Activeloop\Langchain_and_Vector_Databases_in_Production\6_Giving_memory_to_LLMs\rest-api-server-java-sdk\.git\index: Error loading C:\Users\SACHENDRA\Documents\Activeloop\Langchain_and_Vector_Databases_in_Production\6_Giving_memory_to_LLMs\rest-api-server-java-sdk\.git\index
Error loading C:\Users\SACHENDRA\Documents\Activeloop\Langchain_and_Vector_Databases_in_Production\6_Giving_memory_to_LLMs\rest-api-server-java-sdk\.git\objects\pack\pack-85516017461ed4c606743dda75f0577c7edd2ef0.idx: Error loading C:\Users\SACHENDRA\Documents\Activeloop\Langchain_and_Vector_Databases_in_Production\6_Giving_memory_to_LLMs\rest-api-server-java

In [5]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
splitted_text = text_splitter.split_documents(docs)

Created a chunk of size 1020, which is longer than the specified 1000
Created a chunk of size 1540, which is longer than the specified 1000
Created a chunk of size 1597, which is longer than the specified 1000
Created a chunk of size 1024, which is longer than the specified 1000
Created a chunk of size 1467, which is longer than the specified 1000


In [8]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake

# Before executing the following code, make sure to have
# your OpenAI key saved in the “OPENAI_API_KEY” environment variable.
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# TODO: use your organization id here. (by default, org id is your username)
my_activeloop_org_id = "sachendra"
my_activeloop_dataset_name = "langchain_course_chat_with_gh"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"

db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)
db.add_documents(splitted_text)

Using embedding function is deprecated and will be removed in the future. Please use embedding instead.


Your Deep Lake dataset has been successfully created!


Creating 117 embeddings in 1 batches of size 117:: 100%|█████████████████████████████████| 1/1 [00:40<00:00, 40.87s/it]

Dataset(path='hub://sachendra/langchain_course_chat_with_gh', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype       shape      dtype  compression
  -------    -------     -------    -------  ------- 
   text       text      (117, 1)      str     None   
 metadata     json      (117, 1)      str     None   
 embedding  embedding  (117, 1536)  float32   None   
    id        text      (117, 1)      str     None   


['b612e24a-0dca-11ef-993a-a841f4293306',
 'b612e24b-0dca-11ef-96cb-a841f4293306',
 'b612e24c-0dca-11ef-83b5-a841f4293306',
 'b612e24d-0dca-11ef-981c-a841f4293306',
 'b612e24e-0dca-11ef-ad30-a841f4293306',
 'b612e24f-0dca-11ef-8956-a841f4293306',
 'b612e250-0dca-11ef-add1-a841f4293306',
 'b612e251-0dca-11ef-82e9-a841f4293306',
 'b612e252-0dca-11ef-b5bd-a841f4293306',
 'b612e253-0dca-11ef-a4d5-a841f4293306',
 'b612e254-0dca-11ef-866f-a841f4293306',
 'b612e255-0dca-11ef-ba2f-a841f4293306',
 'b612e256-0dca-11ef-8806-a841f4293306',
 'b612e257-0dca-11ef-8d9d-a841f4293306',
 'b612e258-0dca-11ef-99b0-a841f4293306',
 'b612e259-0dca-11ef-873c-a841f4293306',
 'b612e25a-0dca-11ef-a17c-a841f4293306',
 'b612e25b-0dca-11ef-b9f0-a841f4293306',
 'b612e25c-0dca-11ef-ba87-a841f4293306',
 'b612e25d-0dca-11ef-becd-a841f4293306',
 'b612e25e-0dca-11ef-a75d-a841f4293306',
 'b612e25f-0dca-11ef-9353-a841f4293306',
 'b612e260-0dca-11ef-9dbe-a841f4293306',
 'b612e261-0dca-11ef-ab18-a841f4293306',
 'b612e262-0dca-

In [9]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
# Create a retriever from the DeepLake instance
retriever = db.as_retriever()

# Set the search parameters for the retriever
retriever.search_kwargs["distance_metric"] = "cos"
retriever.search_kwargs["fetch_k"] = 100
retriever.search_kwargs["k"] = 10

# Create a ChatOpenAI model instance
model = ChatOpenAI()

# Create a RetrievalQA instance from the model and retriever
qa = RetrievalQA.from_llm(model, retriever=retriever)

# Return the result of the query
qa.run("What is the repository's name?")

C:\Users\SACHENDRA\miniconda3\envs\activeloop2\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'The repository\'s name is "Payment REST API Client SDK."'

Create an Interface:

Creating a user interface (UI) for the bot to be accessed through a web browser is an optional yet crucial step. This addition will elevate your ideas to new heights, allowing users to engage with the application effortlessly, even without any programming expertise. This repository uses the Streamlit platform, a fast and easy way to build and deploy an application instantly for free. It provides a wide range of widgets to eliminate the need for using backend or frontend frameworks to build a web application.

In [10]:
!pip install streamlit streamlit_chat

  Using cached Pillow-9.5.0-cp312-cp312-win_amd64.whl.metadata (9.7 kB)
Using cached Pillow-9.5.0-cp312-cp312-win_amd64.whl (2.5 MB)
  Attempting uninstall: pillow
    Found existing installation: pillow 10.2.0
    Uninstalling pillow-10.2.0:
      Successfully uninstalled pillow-10.2.0


  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
deeplake 3.9.4 requires pillow~=10.2.0, but you have pillow 9.5.0 which is incompatible.


In [14]:
import streamlit as st
from streamlit_chat import message

# Set the title for the Streamlit app
st.title(f"Chat with GitHub Repository")

# Initialize the session state for placeholder messages.
if "generated" not in st.session_state:
	st.session_state["generated"] = ["i am ready to help you ser"]

if "past" not in st.session_state:
	st.session_state["past"] = ["hello"]

# A field input to receive user queries
# user_input = st.text_input("", key="input")

# A field input to receive user queries
user_input = st.text_input("Enter your query:", key="input")

# Search the databse and add the responses to state
if user_input:
	output = qa.run(user_input)
	st.session_state.past.append(user_input)
	st.session_state.generated.append(output)

# # Create the conversational UI using the previous states
# if st.session_state["generated"]:
# 	for i in range(len(st.session_state["generated"])):
# 		message(st.session_state["past"][i], is_user=True, key=str(i) + "_user")
# 		message(st.session_state["generated"][i], key=str(i))

# Create the conversational UI using the previous states
if "generated" in st.session_state:
    for i in range(len(st.session_state["generated"])):
        message(st.session_state["past"][i], is_user=True, key=str(i) + "_user")
        message(st.session_state["generated"][i], key=str(i))

In [16]:
!streamlit run ./chat.py

Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: ./chat.py
